In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
pretrain_features = {
    "learning_rate": 0.1,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [4]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    
#     scaler = StandardScaler()
#     x_pretrain = scaler.fit_transform(x_pretrain)
#     x_train = scaler.transform(x_train)
#     x_test_transed = scaler.transform(x_test)
#     x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [5]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

In [6]:
class AE(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1000, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.01)
            )

        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(512, 1000),
            nn.BatchNorm1d(1000),
            nn.LeakyReLU(0.01)
            )
            
        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [7]:
eval_size = 1000
batch_size = 256
learning_rate = 0.01
ae_model = AE()
ae_model.train()
ae_model.to(device)

def train_autoencoder():
    x_tr, x_val, y_tr, y_val = train_test_split(x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(ae_model.parameters(), lr=learning_rate)
    # optimizer = torch.optim.SGD(ae_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 1000
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, _] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, _] in val_loader:
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        
        # if(epoch % 10 == 0):
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_autoencoder()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: train loss: 0.0469012905122066, val loss: 0.031096543431282042
Epoch 2: train loss: 0.02780783176695814, val loss: 0.025113112807273866
Epoch 3: train loss: 0.02329130178264209, val loss: 0.03144400544464588
Epoch 4: train loss: 0.019720219261488137, val loss: 0.01859737168252468
Epoch 5: train loss: 0.01733484455152434, val loss: 0.016893097937107086
Epoch 6: train loss: 0.015799177887488386, val loss: 0.015091578602790832
Epoch 7: train loss: 0.014519363838342987, val loss: 0.014138679437339307
Epoch 8: train loss: 0.0136059241332874, val loss: 0.013433186240494251
Epoch 9: train loss: 0.012916220368779435, val loss: 0.012732396207749843
Epoch 10: train loss: 0.012306574097397376, val loss: 0.01217357062548399
Epoch 11: train loss: 0.011893344141694965, val loss: 0.011822087474167346
Epoch 12: train loss: 0.011431326527072459, val loss: 0.01122866529226303
Epoch 13: train loss: 0.011053578493090308, val loss: 0.010876182943582535
Epoch 14: train loss: 0.010694082257546941, v

In [16]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
        # and then used to extract features from the training and test data.
        self.seq = nn.Sequential(
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.6),
            nn.Linear(64, 1)
            )


    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture 
        # defined in the constructor.
        x = self.seq(x)
        return x
    
    def encode(self, x):
        x = self.seq[:-3](x)
        return x

In [9]:
aed_x_pretrain = ae_model.encode(torch.tensor(x_pretrain, dtype=torch.float).to(device)).detach().cpu().numpy()

In [17]:
# model declaration
nn_model = Net()
nn_model.to(device)
nn_model.train()
batch_size = 256
eval_size = 1000
learning_rate = 0.05
weight_decay = 0.0001

def train_nn():
    x_tr, x_val, y_tr, y_val = train_test_split(aed_x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set 
    # to monitor the loss.
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 500
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = nn_model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = nn_model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_nn()

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1: train loss: 0.5522916644422375, val loss: 0.097227508187294
Epoch 2: train loss: 0.06730460220271227, val loss: 0.05047814780473709
Epoch 3: train loss: 0.04896574281247295, val loss: 0.04429624062776565
Epoch 4: train loss: 0.04463283661555271, val loss: 0.0445116383433342
Epoch 5: train loss: 0.041710337719442894, val loss: 0.03859437975287437
Epoch 6: train loss: 0.03938714456071659, val loss: 0.03859195524454117
Epoch 7: train loss: 0.03886460914569242, val loss: 0.04096921890974045
Epoch 8: train loss: 0.038402497321975475, val loss: 0.04493274086713791
Epoch 9: train loss: 0.03774038075488441, val loss: 0.036222328051924706
Epoch 10: train loss: 0.03642653064885918, val loss: 0.035131525963544845
Epoch 11: train loss: 0.037953362908898565, val loss: 0.04176260051131248
Epoch 12: train loss: 0.03550053739791014, val loss: 0.034019233465194705
Epoch 13: train loss: 0.03675932344490168, val loss: 0.03461115227639675
Epoch 14: train loss: 0.035345912370754747, val loss: 0.03

In [18]:
from sklearn.linear_model import LinearRegression
def get_regression_model(X, y):
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    model = LinearRegression()
    model.fit(X, y)
    return model

In [19]:
result_file = "results-ae-nn-lr.csv"

In [24]:
featured_x_train = ae_model.encode(torch.tensor(x_train, dtype=torch.float).to(device))
featured_x_train = nn_model.encode(featured_x_train).detach().cpu().numpy()
scaler = StandardScaler()
featured_x_train = scaler.fit_transform(featured_x_train)

y_pred = np.zeros(x_test.shape[0])
featured_x_test = nn_model.encode(ae_model.encode(torch.tensor(x_test.to_numpy(), dtype=torch.float).to(device)))
featured_x_test = scaler.transform(featured_x_test.detach().cpu().numpy())
# featured_x_test = torch.tensor(featured_x_test, dtype=torch.float).to(device)
# y_pred = one_model(featured_x_test).squeeze(-1).detach().cpu().numpy()
lr = get_regression_model(featured_x_train, y_train)
print(lr.score(featured_x_train, y_train))
y_pred = lr.predict(featured_x_test)

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

0.8742937044897616
Predictions saved to results-ae-nn-lr.csv, all done!
